# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
19/08/2024

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import cv2
import os
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# des beaux graphiques
import seaborn as sns

from fonctions_raideur_poncutelle import *


In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x0000026908996350>
Using matplotlib backend: TkAgg


# mesures finales

In [3]:
# SENSIBILITE CAPTEUR
S_force_bs = 0.04026/100      # bs pour banc statique, sensibilité en N/mV
G_force_bs = 900        # gain du banc de mesure
S_camera = 98  # pix/mm

dossier = 'mesures_BS_90anches_2024.10.17'
imag = '/image_'
seuil = 120

# LOAD DATAs
nbre_anche = 1


In [4]:
# Profils de raideurs 

plt.close('all')

# with open(f'2024.05.30/data_BS_force.txt', 'w') as file:pos_y = np.linspace(start, end, num=100)
with open(f'data_BS_force1000.txt', 'w') as file:
    file.write(f'Num_Anche force_constructeur K0 K1 x0 Delta_K K00 K01 x00 \n')
    coef_as =[]
    coef_bs = []
    coef_raideurs = []
    coef_origines = []
    labels = open(dossier +'/data_anche1.txt').readlines()[0].strip()
    # print(labels)
    # for i in range (nbre_anche):
    for i,anc in enumerate( [1,2]):
        y_centre = open(dossier + f'/data_anche{i+1}.txt').readlines()[4].strip() 
        data = np.genfromtxt(dossier + f'/data_anche{i+1}.txt', delimiter=' ', skip_header=5)
        force_constructeur = data[:,0]
        position_y = data[:,2]
        position_z = data[:,3]
        forces = data[:,5]
        
        coef_raideur, coef_origine, Delta_K, _, pos_y, coef_a, coef_b = modelisation_profile_de_raideur(position_y, position_z, forces, numero_anche=f'{i+1}', profil_only=True, pentes_et_profils=False)
        linspace_posy = np.linspace(pos_y[0], pos_y[-1], num=100)
        coef_as.append(coef_a) # forme développée
        coef_bs.append(coef_b)
        coef_raideurs.append(coef_raideur) # forme canonique
        coef_origines.append(coef_origine)
        
        file.write(f'{i+1} {force_constructeur[0]} {coef_raideur[0]} {coef_raideur[1]} {coef_raideur[2]} {Delta_K} {coef_origine[0]} {coef_origine[1]} {coef_origine[2]} \n')
        # print(coef_raideur, coef_origine)
       
    # a pour la raideur
    # b pour l'ordonnée à l'origine    
    # moyennes et erreurs
    mean_a = np.mean(coef_as, axis=0)
    MSE_a = np.sqrt(np.mean((coef_as - mean_a)**2, axis=0)) # erreur quadratique moyenne 
    sigma_a = np.std(coef_as, axis=0) # STD
    N_lib = 5-1 # degré de liberté dans la loi de student = nbre_repetition -1
    tau_confiance = 0.95 # on choisit d'avoir un taux de confiance de 95%
    t_student = 2.78
    
    mean_b = np.mean(coef_bs, axis=0)
    MSE_b = np.sqrt(np.mean((coef_bs - mean_b)**2, axis=0))
    sigma_a = np.std(coef_as, axis=0)
    
   

## Image

In [ ]:
plt.close('all')
seuil = 110
S_camera = 98  # pix/mm
G_force_bs = 900        # gain du banc de mesure
S_force = (0.04026*10**3)/100 

# Initialisation sur une image
# Load data
data = np.genfromtxt(dossier + '/data_anche1.txt', delimiter=' ', skip_header=5)
force_constructeur = data[:,0]
position_y = data[:,2]
position_z = data[:,3]
force_brute = data[:,4]
# Lecture de l'image en nuance de gris, mise en noir et blanc
I = cv2.imread(dossier + imag + '1_0_0.jpg', cv2.IMREAD_GRAYSCALE) 
_, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
I = I.astype(np.float32)

# Rogner la photo
xy_anche = bord_anche(I) # relever la position des bords de l'anche "manuellement"
I_rogn = rogner(I, xy_anche) # rogner l'image proche des bords de l'anche
coord_rogn = bord_anche(I_rogn) # relever la position des bords dans l'image rognée

Ys = len(np.unique(position_y))
Zs = len(np.unique(position_z))

# slicing de l'image 
pas_z = 0.05 #mm
nbr_slice = 20
(w, h) = I_rogn.shape

# image de référence horizontale -> anche métalique
ref_horizon = cv2.imread(dossier + '/ref_17.10.jpg', cv2.IMREAD_GRAYSCALE)
_, ref_horizon = cv2.threshold(ref_horizon, 25, 255, cv2.THRESH_BINARY)  
ref_horizon = ref_horizon.astype(np.float32)
ref_horizon =  rogner(ref_horizon, xy_anche)
Xv_ref, Yv_ref = detection_profil(ref_horizon, seuil, coord_rogn) # détection de la pointe de l'anche image par image
Yv_ref_array = np.array(Yv_ref)


# l'anche de référence est moins large qu'une anche, on complète les valeurs inexistantes par une valeure moyenne de Yv
Yv_mean = np.nanmean(Yv_ref_array) 
Yv_ref_array[np.isnan(Yv_ref_array)] = Yv_mean
mask_ref = ~np.isnan(Xv_ref) & ~np.isnan(Yv_ref_array) & ~np.isinf(Xv_ref) & ~np.isinf(Yv_ref_array)
Xv_ref_clean = Xv_ref[mask_ref]
Yv_ref_clean = Yv_ref_array[mask_ref]

Yv_mean = np.mean(Yv_ref_clean)
Yv_std = np.std(Yv_ref_clean)

# Remplacement des valeurs qui s'écartent trop de la moyenne
for i in range(len(Yv_ref_clean)):
    if abs(Yv_ref_clean[i] - Yv_mean) > (2 * Yv_std) :
        Yv_ref_clean[i] = Yv_mean
    # Yv_ref_clean[i] = Yv_ref_clean[i]-100

# valeur en mm        
Xv_ref_physique = Xv_ref_clean/S_camera
Yv_ref_physique = Yv_ref_clean/S_camera
# print(Yv_ref_physique)

# plt.imshow(ref_horizon, cmap='gray')
# plt.plot(Xv_ref_clean, Yv_ref_clean, '-', lw=2)

In [ ]:
plt.close('all')
plot_fig = False
plot_z0 = False
plot_multianches = False # pour 4 anches [0,26,40,80]

if plot_multianches :
    fig, axes = plt.subplots(4, 5, figsize=(15, 30))
    fig.suptitle(f"Position du capteur sur l'axe y (Y_ci)", fontsize=12)

with open( f'data_BS_img_test.txt', 'w') as file:
    file.write(f'Numero_anche force_constructeur A_00 matrice_de_raideur(9x19) matrice_ordonnees_origine(9x19) \n')
    # for idxanche, numero_anche in enumerate([1,2]): # si plot_multianches
    for numero_anche in range(nbre_anche): # pour les autres plots
        matrix_a = np.zeros((Ys,nbr_slice-1))
        matrix_b = np.zeros((Ys,nbr_slice-1))
        # 2 Matrice (pente et ordonnée origine) de la taille du 'nombre de points d'appui du capteurs sur la largeur de l'anche' x 'nombre de slices/tranches/points d'obeservation'
        A = np.zeros((Ys,Zs)) 
        B = np.zeros((Ys,Zs))
        
        # Load data
        data = np.genfromtxt(dossier + f'/data_anche{numero_anche+1}.txt', delimiter=' ', skip_header=5)
        force_constructeur = data[:,0]
        position_y = data[:,2]
        position_z = data[:,3]
        force_brute = data[:,4]
        A_00 = [] # surface initiale entre l'anche de référence et l'anche au repos
        
        # if plot_z0 : 
        #     plt.figure()
        # for idxy, y in enumerate([0,2,4,6,8]): # si plot_multianches
        for y in range(Ys) : # pour les autres plots
            if plot_fig :
                fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            A = np.zeros((Zs+1, (nbr_slice-1)))
            
            for z in range(Zs):
                img_path = dossier + imag + f'{numero_anche+1}_{y}_{z}.jpg' # rechercher toutes les images
                if not os.path.exists(img_path):  # si le chemin n'est pas trouvé, cette image est ignorée et on passe à la suivante
                    continue
                # Lecture image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)   
                _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)  
                img = img.astype(np.float32)
                img_rogn =  rogner(img, xy_anche) # rogner image suivant les coordonées des bords de la 1ere anche      
                
                Xv, Yv = detection_profil(img_rogn, seuil, coord_rogn) # détection de la pointe de l'anche image par image
                Yv_array = np.array(Yv)
                
                # se débarasser des valeurs abérentes (nan et inf dans les Xv et Yv)
                def clean(arr):
                    filled_arr = arr.copy()  # Créer une copie de l'array d'origine
                    for i in range(1, len(filled_arr)):
                        if np.isnan(filled_arr[i]) or np.isinf(filled_arr[i]):
                            filled_arr[i] = filled_arr[i - 1]  # Remplacer par la valeur précédente
                    return filled_arr

                # Remplir les valeurs dans Xv et Yv_array
                Xv_clean = clean(Xv)
                Yv_clean = clean(Yv_array)

                # grandeur physique : on passe de pixel en mm
                Xv_physique = Xv_clean/S_camera
                Yv_physique = Yv_clean/S_camera
                
                # calcul de "l'ouverture au repos"
                if z ==0 :
                    # plt.figure()
                    diff = [0 if m == 0 or r == 0 else (m - r) for m, r in zip(Yv_ref_physique, Yv_physique)]
                    diff = np.abs(diff)
                    diff = np.array(diff)
                    
                    # Vérifier si les longueurs sont différentes
                    if len(diff) != len(Xv_physique):
                        # Interpoler diff pour qu'il corresponde à la taille de Xv
                        Xv_interp = np.linspace(Xv_physique[0], Xv_physique[-1], len(diff))
                        diff_interp = np.interp(Xv_interp, np.linspace(Xv_physique[0], Xv_physique[-1], len(diff)), diff)
                    else:
                        diff_interp = diff
                        
                    # Calculer l'aire avec les valeurs interpolées
                    A_00.append(np.trapz(diff_interp, Xv_physique))
                    
                    # représenter l'ouverture au repos
                    if plot_z0 :
                        height, width = img_rogn.shape
                        width_mm = width  / S_camera
                        height_mm = height  / S_camera
                        # plt.imshow(img_rogn, cmap='gray', extent=[0, width_mm, 0, height_mm], origin='lower', alpha=0.2)
                        # plt.imshow(img_rogn, cmap='gray')
                        plt.plot(Xv_physique, Yv_physique, '-', color='#273877', lw=2, label=f'Profil inital de l\'anche {numero_anche} \n(sans déformation)')
                        # f'$y_{y+1}$ = {np.unique(position_y)[y]}'
                        plt.plot(Xv_physique, Yv_ref_physique, '-', color='#E5442F', lw=2, label='Anche de référence')
                        plt.fill_between(Xv_physique, Yv_physique, Yv_ref_physique, where=(Yv_physique > Yv_ref_physique), interpolate=True, color='#2EB1D5', alpha=0.3, label='Ouverture d\'anche')
                        plt.fill_between(Xv_physique, Yv_physique, Yv_ref_physique, where=(Yv_physique < Yv_ref_physique), interpolate=True, color='#2EB1D5', alpha=0.3)
                        plt.xlabel('y (mm)')
                        plt.ylabel('z (mm)')
                        plt.grid()
                        plt.legend(title='')
                        # plt.legend(loc="upper left", bbox_to_anchor=(1, 1), title='pointe de l\'anche à force et déplacement nuls : \n$F=0, z=0$')
                
                
                # Etude de la pointe d'anche par tranche
                Xv_slice_center = [] 
                Yv_mean_per_slice = []
                slice_size = (np.abs(Xv_clean[-1]-Xv_clean[0]))/nbr_slice   # taille en pixel d'une tranche
                anche_center = (np.abs(Xv_clean[-1]-Xv_clean[0]))/2 
                for n in range(nbr_slice-1):
                    # début et fin de chaque slice par itération            
                    slice_left = int(Xv_clean[0]+n*slice_size)
                    slice_right = int(Xv_clean[0]+(n+1)*slice_size)
                    Xv_slice_center.append(-anche_center + slice_left + np.abs(slice_left-slice_right)/2) # coordonnée centrale de la tranche
                    Yv_slice = Yv_clean[slice_left:slice_right]
                    Yv_mean_per_slice.append(np.mean(Yv_slice)) # déplacement moyen par slice (moyenne sur les Yv)
                
                # grandeur physique mm
                Xv_slice_center_phys = np.array(Xv_slice_center)/S_camera
                Yv_mean_per_slice_phy = np.array(Yv_mean_per_slice)/S_camera   
                # print(Yv_mean_per_slice_phy.shape)  
                # print(Yv_mean_per_slice_phy)  
                # data[z,:]=Yv_mean_per_slice_phy
                A[0,:] = Xv_slice_center_phys
                A[z+1,:] = Yv_mean_per_slice_phy
                
                
                # représentation de l'image avec contour, le déplacement seul et les souplesses
                if plot_fig :
                    height, width = img_rogn.shape
                    width_mm = width  / S_camera
                    height_mm = height  / S_camera
                    axs[0].imshow(img_rogn, cmap='gray', extent=[0, width_mm, 0, height_mm], origin='lower')
                    # axs[0].imshow(img_rogn, cmap='gray')
                    axs[0].plot(Xv_physique, Yv_physique, '-', lw=2)
                    axs[0].invert_yaxis()
                    axs[0].set_xlabel('y (mm)')
                    axs[0].set_ylabel('z (mm)')
                    axs[1].plot(Xv_slice_center_phys, Yv_mean_per_slice_phy, '.-', lw=2, label=f'{np.unique(position_z[z])}')
                    axs[1].set_xlabel('y (mm)')
                    axs[1].set_ylabel('z (mm)')
                    # axs[1].legend(title='position z d\'appui (mm)')
                    axs[1].grid('True')  

    
                # représentation des déplcaments pour plusieurs anches
                if plot_multianches :
                    axes[idxanche, idxy].plot(Xv_slice_center_phys, Yv_mean_per_slice_phy, '.-', lw=2, label=f'{np.unique(position_z[z])}')
                    axes[idxanche, idxy].set_xlabel('y (mm)')
                    axes[idxanche, idxy].set_ylabel('z (mm)')
                    # axs[1].legend(title='position z d\'appui (mm)')
                    axes[idxanche, idxy].grid('True')  
                    axes[0, idxy].set_title(f"$i={y}$", fontsize=12, pad=20)
                    axes[idxanche, 0].set_ylabel(f"Anche n°{numero_anche+1}", fontsize=12, rotation=0, labelpad=60, ha='right')
                    


            forces = np.array((force_brute-min(force_brute))/(S_force*G_force_bs))

            # plt.figure()
            pentes = [] # initialisation du vecteur contenant les pentes
            ordos_origine = []
            coefficients_a = []
            coefficients_b = []
            mymap = plt.get_cmap("Blues")
            for idx, i in enumerate(range(len(Xv_slice_center))):
                Fs=np.zeros((len(Xv_slice_center),4))
                for j in range(0, len(forces), 4):
                    Fs[i,:] = np.array(forces[j:j+4])
                Y = A[1:,i]
                # print(Fs[i,:].shape)
                coefficient = np.polyfit(Fs[i,1:], Y[1:], 1)
                couleur = mymap((idx+4) / len(np.unique(position_y)))
                # plt.plot(Fs[i,:], Y, '.', label=f'y = {i} mm', color=couleur)
                # plt.plot(Fs[i,1:], np.polyval(coefficient, Fs[i,1:]), linestyle='-', color=couleur, label=f'z = {np.round(coefficient[0], 2)}x + {np.round(coefficient[1], 2)}')
                # plt.xlabel(f'Force mesurée au point d\'appuie du capteur {np.unique(position_y)[y]} en N')
                # plt.ylabel('Déplacement mesuré par voix optique en mm')
                # plt.legend(bbox_to_anchor=(-0.5, 0.5), loc='center left', fontsize=10, borderaxespad=0)
                # plt.grid('True')
                if plot_fig :
                    axs[2].plot(Fs[i,:], Y, '.', label=f'y = {i} mm', color=couleur)
                    axs[2].plot(Fs[i,1:], np.polyval(coefficient, Fs[i,1:]), linestyle='-', color=couleur, label=f'z = {np.round(coefficient[0], 2)}x + {np.round(coefficient[1], 2)}')
                    axs[2].set_xlabel(f'Force mesurée (N)')
                    axs[2].set_ylabel('Déplacement z (mm)')
                    # axs[2].legend(bbox_to_anchor=(-0.5, 0.5), loc='center left', fontsize=10, borderaxespad=0)
                    axs[2].grid('True')
                coefficients_a.append(coefficient[0])
                coefficients_b.append(coefficient[1])
            
            if plot_multianches :    
                matrix_a[idxy,:] = coefficients_a
                matrix_b[idxy,:] = coefficients_b
            
            if plot_fig :
                matrix_a[y,:] = coefficients_a
                matrix_b[y,:] = coefficients_b
            
        flat_matrix_a = matrix_a.flatten()
        flat_matrix_b = matrix_b.flatten()
        # print(matrix_b.shape)
        # print(len(flat_matrix_b))
        flat_matrix_tot = np.concatenate((flat_matrix_a, flat_matrix_b))
        flat_matrix_tot = ' '.join(map(str, flat_matrix_tot))
        # print(len(flat_matrix_tot))
        A_00_mean = np.mean(A_00)   
        A_00_std = np.std(A_00)     
        # print(A_00, A_00_mean)
                
        file.write(f'{numero_anche+1} {force_constructeur[0]} {A_00_mean} {flat_matrix_tot} \n')  # Écrit les matrices aplaties sur une seule ligne
     